# Mathe2 Minitest 4

# Code

In [1]:
import sympy as sp
from functools import partial
from urllib.parse import quote
from typing import Callable, Iterable
from functools import singledispatchmethod
from IPython.core.display import Math, Latex, Markdown

# - - - - - - - - -

###############
### Globals ###
###############

k = sp.symbols('k', positive = True)
x = sp.symbols('x')


# shorthand

def L(Obj, *args, **kwargs) -> str: return sp.latex(Obj, *args, **kwargs)

def P(STR:str, *args, **kwargs):        display(Math(STR, *args, **kwargs))
def PL(STR:str, *args, **kwargs):       display(Latex(STR, *args, **kwargs))
def PM(STR:str, *args, **kwargs):       display(Markdown(STR, *args, **kwargs))

def PF(*args, Format:str | None = None):
   LF = lambda O: L(O) if not isinstance(O, str) else O # Latex-Filter
   Format = " ".join(len(args)*["{}"]) if Format is None else Format
   P(Format.format(*map(LF, args)))


## Sympy

### Aliases
from sympy import Rational as R, oo

# - - - - - - - - -

######################
### Shared Library ###
######################

## Latex

def LSum(Seq:sp.Expr, 
   upper:sp.Expr=oo, lower:sp.Expr=sp.Number(0), var = k
   ) -> Callable[..., str]:
   def Sum(upper:sp.Expr | int = upper, lower:sp.Expr=lower,
           SUM:str = r"\sum_{{{} = {}}}^{{{}}}",
           SumFormat:Callable[[str, str], str] = lambda S, sequence: f"{S} {sequence}"
           )->str:
      return SumFormat(SUM.format(*map(L, [var, lower, upper])), L(Seq))
   return Sum

def L_limit(Seq:sp.Expr, seek:sp.Expr=oo, var = k, Format = "{} {}") -> str:
   LIMIT = r"\lim_{{{} \to {}}}".format(L(var), L(seek))
   return Format.format(LIMIT, L(Seq))

def LABS_limit(Seq:sp.Expr, seek:sp.Expr=oo, var = k) -> str:
   return L_limit(Seq, seek, var, Format = "{} |{}|")

def LABS_limitEval(Seq:sp.Expr, seek:sp.Expr=oo, var = k) -> str:
   LIMIT = r"\lim_{{{} \to {}}}".format(L(var), L(seek))
   return f" \
   {LABS_limit(Seq, seek, var)} = \
   {LIMIT} {L(abs(Seq).doit())} = \
   {L(abs(Seq).limit(var, seek))} \
   "

def L_SeriesDisplay(Sequence:list) -> str:
   return f"{'+'.join(f'({L(a)})' for a in Sequence)} = {L(sum(Sequence))}"

def L_SequenceDisplay(Sequence:list) -> str:
   return r'\{' + (sep := r',\hspace{.5cm}').join(L(a) for a in Sequence) + f"{sep}..." + r'\}'


# - - - - - - - - - -

## Wolfram Alpha

def WRALink(CMD:str)->str:
  return r"https://www.wolframalpha.com/input/?i=" + quote(CMD)

def WRAMarkdown(
      CMD:str, 
      Format:str = "#### [{}]({})",
      FormatLink:Callable[[str], str] = lambda CMD: f"WRA: {CMD}"
      ):
  PM(Format.format(FormatLink(CMD), WRALink(CMD)))

WRAMarkdownMath = partial(WRAMarkdown, FormatLink = lambda CMD: f"WRA: ${CMD}$")

# - - - - - - - - - - - - -

## Math

class SerializedSequence:
   def __init__(
      self, Seq:sp.Expr, var:sp.Symbol = k, 
      Sub: Callable[[int|sp.Expr], sp.Expr] | None = None
      ) -> None:
      self.Seq = Seq
      self.k = var
      if Sub is None:
        def Sububsitude(x, Seq:sp.Expr = self.Seq) -> sp.Expr:
           return Seq.subs({self.k: x}) # type: ignore
        Sub = Sububsitude
      self.Sub = Sub

   @staticmethod
   def numerical(index:slice) -> Iterable:
      start = 0 if index.start is None else index.start
      stop = index.stop     # must be there!
      step  = 1 if index.step is None else index.step
      return range(start, stop+1, step)   # Series start at 0 too

   @singledispatchmethod      # general dispatch definition DEFAULT
   def __getitem__(self, index):
      raise NotImplementedError('Unsupported Access')

   @__getitem__.register(int)    # Normal Sequence Behaviour
   def _(self, index) -> sp.Expr:
      return self.Sub(index)

   @__getitem__.register(slice)  # A Series
   def _(self, index):
      if index.stop is None:    # Series limit to inf #TODO
         # regardless what the other values are! start, stop, step
         raise NotImplemented("Still don't know how to do it")
      if index.stop is oo:      # Sequence limit to inf
         return self.Seq.limit(k, oo)
      if index.start is None:
         return sum(self.Sub(i) for i in self.numerical(index)) # type: ignore
      return [self.Sub(i) for i in self.numerical(index)]

def GeometricSeries(Sequence:sp.Expr, var:sp.Symbol = k) -> sp.Expr:
   # should be a Nullfolge Anyway #Unfug
   return (1 - abs(Sequence))/(1 - Sequence.subs(var, 1))  # type: ignore

## Taylor Polynomials

KnownSeries:dict[str, SerializedSequence] = dict()

def SubstitutedSeries(
    Sequence:SerializedSequence,
    f:sp.Expr = sp.Function('f'),
    x:sp.Symbol = x,
    innerFunction: sp.Expr = x,
    factor:sp.Expr = sp.Number(1)
    )->SerializedSequence:
    def evaluate(i:int|sp.Expr) -> sp.Expr:
        return factor * Sequence.Sub(i).subs(x, innerFunction)
    return SerializedSequence(f, Sub=evaluate)

# beware of empty space
KnownSeries['1/(1 - x)'] = SerializedSequence(
   x**k, var=k
)

BruchSeries = partial(SubstitutedSeries, KnownSeries['1/(1 - x)'])

# - - - -

KnownSeries['e'] = SerializedSequence(
   x**k / sp.factorial(k), var=k
)

ESeries = partial(SubstitutedSeries, KnownSeries['e'])

# - - - -

# Buch starts at 1 *or modify to start at 0
KnownSeries['ln(1 + x)'] = SerializedSequence(
   (-1)**k * x**(k+1) / (k+1), var=k
)

LNSeries = partial(SubstitutedSeries, KnownSeries['ln(1 + x)'])

# - - - -

KnownSeries['sin'] = SerializedSequence(
   (-1)**k / sp.factorial(2*k+1) * x**(2*k + 1),
   var=k
)

SinSeries = partial(SubstitutedSeries, KnownSeries['sin'])

# - - - -

KnownSeries['cos'] = SerializedSequence(
   (-1)**k / sp.factorial(2*k) * x**(2*k),
   var=k
)

CosSeries = partial(SubstitutedSeries, KnownSeries['cos'])

# - - - - - - -- - - - -- - - - -

def PolynomialApproximationOf(Sequence:SerializedSequence, until:int, x:sp.Symbol = x):
    Seq = list()
    def f(i):
        Seq.append(Sequence[i])
        return sp.Poly(Seq[-1], x).degree()

    i = 0
    while f(i) < until:
        i += 1

    return sum(Seq[:-1])

def ApproximateFPolynomial(f:sp.Expr, x0 = 1, until:int = 2, x:sp.Symbol = x):
   PF(f"f ({x}) = ", f)
   def evaluate(i:int | sp.Expr):
      return f.diff(x, i).subs(x, x0)

   DF = f'f{{}} ({x0}) = '
   df = lambda i: DF.format(i*"'")

   for i in range(until+1):
      PF(df(i), evaluate(i))

   def Sub(i:int | sp.Expr) -> sp.Expr:
      return evaluate(i) * (x-x0)**i / sp.factorial(i)

   def Format():
      T = SerializedSequence(f, Sub=Sub)
      return '+'.join(f"{L(t)}" for t in T[0:until][::-1] if t != 0)

   PF(f"T_{{{until}}} ({x}) = ", Format())

### Convergence

def LeibnizK(Seq:sp.Expr, n:int=3, Section:str = '#', k:sp.Symbol = k):
   PM(f"{Section}# Leibniz-Kriterium")

   Sequence = SerializedSequence(Seq)
   P(f"a_{{k}} = {L(Seq)} = {L_SequenceDisplay(Sequence[0:n+2])}")

   P(f"{LABS_limitEval(Seq)}")

   if ((ak := abs(Seq)).limit(k, oo) == 0):
      PM(r" $\implies$ Nullfolge")
   else:
      PM(r" $\implies$ Keine Nullfolge")

   P(f" |a_{{k}} | = {L(ak)} \\hspace{{1cm}} |a_{{k+1}}| = {L(ak.subs(k, k+1).simplify())}")

   def SD(k):   return L_SeriesDisplay(Sequence[0:k])

   P(f"\\widetilde{{S}} = {(Sum := LSum(Seq))(n-1)} = {SD(n-1)}")
   P(f"{Sum(n)} = {SD(n)}")

   # C = lambda a, b: '==' if a == b else ('>' if a > b else '<')
   C = lambda a, b: r'\le' if a <= b else '>'

   P(f"|S_{{{n}}} - \\widetilde{{S}}| =  |{Sum(n)} - {Sum(n-1)}| = \
     |{L(S := Sequence[:n])} - {L(S_ := Sequence[:n-1])}| = \
     {L(Diff := abs(S-S_))} {C(Diff, Sn := abs(Sequence[n]))} ( |a_{{{n}}}| = {L(Sn)})\
      ")

   PM(f"{Section}## Wolfram Alpha")
   WRAMarkdown(CMD=f"|{L(Seq)}| where {k} is {n}", FormatLink=lambda CMD: f"|${L(Seq)}$| where {k} is {n}")
   WRAMarkdownMath(Sum(n-1))
   WRAMarkdownMath(Sum(n))


# - - - - - - - - - - - - -

## Display

def DescribeSequence(Seq:sp.Expr, n:int=4, ParentSection:str = '#'):

   Sequence = SerializedSequence(Seq)

   PM(f"{ParentSection}# Describing Sequence ${L(Seq)}$:")

   for i in range(n):
      PF(k,f"= {i} :",  r"\hspace{1cm}", f"a_{{{i}}} = ", Sequence[i])

   P(f"{L(Seq)} = {L_SequenceDisplay(S := Sequence[0:n+2])}")
   AbsSeqD = L_SequenceDisplay([abs(a) for a in S])
   P(f"|{L(Seq)}| = {L(abs(Seq).doit())} = {AbsSeqD}")
   P(LABS_limitEval(Seq))

   PM(f"{ParentSection}## Wolfram Alpha")
   WRAMarkdownMath(L(Seq))
   WRAMarkdownMath(LABS_limit(Seq))

def DescribeSeries(Seq:sp.Expr, n:int=4, ParentSection:str = '#'):

   Sequence = SerializedSequence(Seq)

   def S(k):   return L_SeriesDisplay(Sequence[0:k])

   PM(f"{ParentSection}# Describing Series ${(Sum := LSum(Seq))()}$:")

   for i in range(n):
      P(f"k = {i}: {Sum(i)} = {S(i)}")

   P(f"{L(Seq)} = {L_SequenceDisplay(Sequence[0:n])}")
   AbsSeqD = L_SequenceDisplay([abs(a) for a in Sequence[0:n]])
   P(f"|{L(Seq)}| = {L(abs(Seq).doit())} = {AbsSeqD}")
   P(LABS_limitEval(Seq))

   PM(f"{ParentSection}## Wolfram Alpha")
   for i in range(n):
      WRAMarkdownMath(Sum(i))
   WRAMarkdownMath(Sum())

def DescribeGeometricSeries(Seq:sp.Expr, n:int=4, ParentSection:str = '#'):

   Sequence = SerializedSequence(Seq)

   def S(k):   return L_SeriesDisplay(Sequence[0:k])

   PM(f"{ParentSection}# Describing Geometric Series ${(Sum := LSum(Seq))()}$:")

   P(f"{L(Seq)} = {L_SequenceDisplay(Sequence[0:n+2])}")
   P(f"k = {(i := n-1)}: {Sum(i)} = {S(i)}")

   series = GeometricSeries(Seq)

   PF(Seq, LSum(Seq)(), L_limit(series), series.limit(k, oo),
      Format="{}, {} = {} = {}")

   PM(f"{ParentSection}## Wolfram Alpha")
   WRAMarkdownMath(Sum())

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Q-1

In [2]:
def A1(Seq:sp.Expr, n:int=4, Section:str = '#'):
   return DescribeSequence(Seq, n, Section)

Sequence = 7/(k+1)
A1(Sequence, 4)

print("Die Reihe ist aber größer als die harmonische Reihe für denselben k wert")
print("Harmonische Reihe -> divergiert ==> Meine Reihe -> divergiert")

## Describing Sequence $\frac{7}{k + 1}$:

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

### Wolfram Alpha

#### [WRA: $\frac{7}{k + 1}$](https://www.wolframalpha.com/input/?i=%5Cfrac%7B7%7D%7Bk%20%2B%201%7D)

#### [WRA: $\lim_{k \to \infty} |\frac{7}{k + 1}|$](https://www.wolframalpha.com/input/?i=%5Clim_%7Bk%20%5Cto%20%5Cinfty%7D%20%7C%5Cfrac%7B7%7D%7Bk%20%2B%201%7D%7C)

Die Reihe ist aber größer als die harmonische Reihe für denselben k wert
Harmonische Reihe -> divergiert ==> Meine Reihe -> divergiert


# Q-2

In [3]:
def A2(Seq:sp.Expr, n:int=4, Section:str = '#'):
    DescribeSequence(Seq)
    DescribeGeometricSeries(Seq)

Sequence = (-R(1, 5))**k

A2(Sequence, 4)

## Describing Sequence $\left(- \frac{1}{5}\right)^{k}$:

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Wolfram Alpha

#### [WRA: $\sum_{k = 0}^{\infty} \left(- \frac{1}{5}\right)^{k}$](https://www.wolframalpha.com/input/?i=%5Csum_%7Bk%20%3D%200%7D%5E%7B%5Cinfty%7D%20%5Cleft%28-%20%5Cfrac%7B1%7D%7B5%7D%5Cright%29%5E%7Bk%7D)

# Q-3

In [4]:
def A3(Seq:sp.Expr, n:int=3, Section:str = '#', k:sp.Symbol = k):
    DescribeSequence(Seq, n)
    LeibnizK(Seq, n)

Sequence = (-1)**k *(k + 3) / (k**2 + 7*k + 6)

A3(Sequence, 3)

## Describing Sequence $\frac{\left(-1\right)^{k} \left(k + 3\right)}{k^{2} + 7 k + 6}$:

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

### Wolfram Alpha

#### [|$\frac{\left(-1\right)^{k} \left(k + 3\right)}{k^{2} + 7 k + 6}$| where k is 3](https://www.wolframalpha.com/input/?i=%7C%5Cfrac%7B%5Cleft%28-1%5Cright%29%5E%7Bk%7D%20%5Cleft%28k%20%2B%203%5Cright%29%7D%7Bk%5E%7B2%7D%20%2B%207%20k%20%2B%206%7D%7C%20where%20k%20is%203)

#### [WRA: $\sum_{k = 0}^{2} \frac{\left(-1\right)^{k} \left(k + 3\right)}{k^{2} + 7 k + 6}$](https://www.wolframalpha.com/input/?i=%5Csum_%7Bk%20%3D%200%7D%5E%7B2%7D%20%5Cfrac%7B%5Cleft%28-1%5Cright%29%5E%7Bk%7D%20%5Cleft%28k%20%2B%203%5Cright%29%7D%7Bk%5E%7B2%7D%20%2B%207%20k%20%2B%206%7D)

#### [WRA: $\sum_{k = 0}^{3} \frac{\left(-1\right)^{k} \left(k + 3\right)}{k^{2} + 7 k + 6}$](https://www.wolframalpha.com/input/?i=%5Csum_%7Bk%20%3D%200%7D%5E%7B3%7D%20%5Cfrac%7B%5Cleft%28-1%5Cright%29%5E%7Bk%7D%20%5Cleft%28k%20%2B%203%5Cright%29%7D%7Bk%5E%7B2%7D%20%2B%207%20k%20%2B%206%7D)

# Q-4

In [5]:
def A4(f:sp.Expr, x:sp.Expr = x, k:sp.Expr = k, ParentSection:str ='#'):
   PF(f"f({L(x)}) = ", f)
   n, d = sp.fraction(f)
   Coef = sp.Poly(d).coeffs()
   factor = R(n,Coef[1])
   q = -R(*Coef)
   PF(f"f({L(x)}) = ", factor, r"\cdot", f"\\frac{{1}}{{1 - ({L(q)}) \\cdot x}}")
   PM(f"{ParentSection}# Geometrische Reihe mit $q = {L(q*x)}$")
   PF(f"f({L(x)}) = ", f, '=', factor, LSum((q*x)**k)(), 
      '=', LSum(seq := q**k)(SumFormat = lambda S, seq: f"{S} {L(factor)} \\cdot {seq} {L(x**k)}"),
      r'\implies', f"a_{{{L(k)}}} = ", factor, r'\cdot', seq
   )

   PF(f"|q| = ", f"|{L(q*x)}| < 1", r'\implies', r := sp.solve(abs(q*x) < 1), 
      r'\implies', f"|{L(x)}| < {L(rv := abs(r.as_set().args[0]))}",
      r'\implies', f"r = {L(rv)}"
      )

f = 5/(2 - 7*x)

A4(f)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Geometrische Reihe mit $q = \frac{7 x}{2}$

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Q-5

In [6]:
def A5():
    S = CosSeries(innerFunction=x**2)

    n = 5

    PF("cos(x) = ...", PolynomialApproximationOf(CosSeries(), n))
    PF("cos(x^2) = ...", PolynomialApproximationOf(CosSeries(innerFunction=x**2), 2*n))

    S5 = PolynomialApproximationOf(S, n)

    PF(r"cos(x^2) \approx ", S5)

    I = sp.integrate(S5, x)

    PF("I(x) = ", I)

    sp.integrate(S5, (x, 0, R(2, 5)))

    # https://www.wolframalpha.com/input?i=int+1+-+x%5E4%2F2%2C+x%2C+0%2C+2%2F5
    

A5()   # static

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Q-6

In [7]:
def A6(f, n):
    ApproximateFPolynomial(f, n)

f = sp.ln(x - 1) / (3 -x)

# f https://www.wolframalpha.com/input?i=ln%28x-1%29%2F%283-x%29+where+x+%3D+2

# df https://www.wolframalpha.com/input?i=first+derivative+of+ln%28x-1%29%2F%283-x%29+at+x+%3D+2

# ddf https://www.wolframalpha.com/input/?i=second+derivative+of+ln(x-1)%2F(3-x)+at+x+%3D+2

A6(f, 2)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Q-7

In [8]:
def A7():
    ShowSeq = lambda S: '+'.join(f"({L(s)})" for s in S if s != 0)

    g = SinSeries()

    n = 4

    PF(r"g(x) \approx", ShowSeq(g[0:1]))

    h = ESeries()

    PF(r"h(x) \approx", ShowSeq(h[0:3]))

    S = [c*x**i for i, c in zip(range(n+1), sp.Poly(h[:10]*g[:10]).all_coeffs()[::-1])] # enumerate shortest

    PF(r"f(x) = h(x) \cdot g(x) = ", ShowSeq(S), '...')

A7()   # static

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Online Resources

## Leibnitz-Kriterium

### [VHM Uni Stuttgart: Folien_Leibniz-Kriterium.pdf](https://vhm.mathematik.uni-stuttgart.de/Vorlesungen/Differentialrechnung/Folien_Leibniz-Kriterium.pdf)